In [22]:
import os
import numpy as np
import lsst.sims.skybrightness as sb
import lsst.sims.ocs
from lsst.sims.utils import altAzPaFromRaDec, ObservationMetaData

In [23]:
field_dict = {}
with open("tessellationFields") as fh:
    field_id = 1
    for line in fh:
        line = line.strip()
        # Skip blank line
        if not line:
            continue
        # Skip comment lines
        if line.startswith("#"):
            continue
        values = (float(x) for x in line.split())
        field = lsst.sims.ocs.configuration.Field(field_id, 3.5, *values)
        field_dict[field_id] = field
        field_id += 1

In [24]:
# Default is LSST observatory and preciseAltAz off but give back LSST filter magnitudes 
sm = sb.SkyModel(mags=True)

In [25]:
mjd = 59581.041666666664
ra = np.array([field.ra for _, field in field_dict.iteritems()])
dec = np.array([field.dec for _, field in field_dict.iteritems()])
obs = ObservationMetaData(mjd=mjd)

alt, az, pa = altAzPaFromRaDec(ra, dec, obs)

In [26]:
alt

array([ 30.34537627,  28.62848452,  30.67382062, ..., -29.23353408,
       -27.18671828, -28.90499397])

In [27]:
alt_mask = np.where(alt > 20.0)
print(alt_mask)
up_ra = ra[alt_mask]
up_dec = dec[alt_mask]
up_alt = alt[alt_mask]
up_az = az[alt_mask]
print(up_alt.size)

(array([   0,    1,    2, ..., 4304, 4312, 4322]),)
1741


In [28]:
sm.setRaDecMjd(up_ra, up_dec, mjd, degrees=True)

In [29]:
# Here are the access patterns to get at the intermediate information calculated by the SkyModel.

# These parameters all have the size of the input RA, DEC list.
# NOTE: All units are radians unless specified.

# Field airmass
print(sm.points["airmass"])
# Field Altitudes
print(sm.points["alt"])
# Field Azimuths
print(sm.points["az"])
# Sun altitude, same for all Fields
print(sm.points["sunAlt"])
# Field Az - Sun Az
print(sm.points["azRelSun"])
# Here Moon altitude is in degrees, same for all Fields
print(sm.points["moonAltitude"])
# Field Az - Moon Az
print(sm.points["azRelMoon"])
# Moon Phase / 100 * 180, same for all Fields
print(sm.points["moonSunSep"])
# Field ecliptic latitude
print(sm.points["altEclip"])
# Field ecliptic longitude - Sun ecliptic longitude
print(sm.points["azEclipRelSun"])

[ 1.98534993  2.09321225  1.966972   ...,  2.865936    2.88272021
  2.90875171]
[ 0.52786436  0.49807424  0.53332067 ...,  0.35642499  0.35425804
  0.35095003]
[        nan  3.10690223  3.19038731 ...,  0.12205063  0.06935841
  0.01680829]
[-0.24299991 -0.24299991 -0.24299991 ..., -0.24299991 -0.24299991
 -0.24299991]
[        nan  5.33273977  5.41622484 ...,  2.34788816  2.29519594
  2.24264582]
[-19.82014005 -19.82014005 -19.82014005 ..., -19.82014005 -19.82014005
 -19.82014005]
[        nan  0.77833498  0.69484991 ...,  2.51999872  2.4673065
  2.41475638]
[ 1.56466008  1.56466008  1.56466008 ...,  1.56466008  1.56466008
  1.56466008]
[-1.16170352 -1.1596319  -1.13548092 ...,  0.33044421  0.35015253
  0.36932067]
[ 6.08607668  5.97973066  6.16755707 ...,  2.44410322  2.39599299
  2.34731659]


In [34]:
# Here are the access patterns to get at the intermediate information calculated by the SkyModel.

# These parameters are all single valued.
# NOTE: All units are radians unless specified.

print(sm.sunAlt)
print(sm.sunAz)
print(sm.moonPhase)
print(sm.moonAlt)
print(sm.moonAz)
print(sm.sunEclipLon)


-13:55:22.3
232:28:08.1
0.86925560236
-19:49:12.5
222:36:27.7
281:17:36.6


In [35]:
mags = sm.returnMags()

In [32]:
# For each RA,DEC you get a ugrizy map
i_filt = 3
bad_count = 0
for ra, dec, alt, az, fmag in zip(up_ra, up_dec, up_alt, up_az, mags):
    i_mag = fmag[i_filt]
    if i_mag == np.Inf:
        bad_count += 1
    print(ra, dec, alt, az, i_mag)
print("Number of bad fields: {} out of {}".format(bad_count, up_ra.size))

(0.0, -90.0, 30.345376266467646, 180.09527155034436, 19.387400475111402)
(180.0, -87.568555000000003, 28.628484517578688, 178.11934388920477, 19.14509064463024)
(324.000429, -87.568550000000002, 30.673820621229673, 182.89104307183246, 19.211965211549639)
(35.999571000000003, -87.568550000000002, 32.740622992961619, 180.57330183382987, 19.311703456870365)
(252.001105, -87.568546999999995, 28.16512868052487, 181.32440563203664, 19.099840081753236)
(107.998895, -87.568546999999995, 31.443857881276394, 177.56835942640558, 19.274674298762111)
(215.99982199999999, -85.272913000000003, 25.685022525096972, 179.22856568825128, 18.969676023166144)
(144.00017800000001, -85.272913000000003, 29.539288072047285, 174.72070079943745, 19.208546742157715)
(0.0, -85.272891999999999, 33.592759707175048, 184.14238846838666, 19.327328896677336)
(288.00050800000002, -85.272852, 28.077125181878454, 184.84562592146685, 19.071122195391442)
(71.999492000000004, -85.272852, 34.548933141812903, 177.53909462061873,